In [1]:
import numpy as np 
import pandas as pd
import statsmodels.api as sm
from sklearn.metrics import explained_variance_score
from sklearn.metrics import r2_score
from sklearn.metrics import median_absolute_error
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_squared_error
from sklearn.pipeline import Pipeline
import seaborn as sns
import matplotlib.pyplot as plt
from matplotlib.ticker import PercentFormatter
import cvxopt
from sklearn import linear_model
from sklearn.preprocessing import PolynomialFeatures

In [2]:
def select_train(list_var, data):
    for var in list_var :
        data.pop(var)
    return data

def select_test(list_var, data):
    for var in list_var :
        data.pop(var)
    return data

In [4]:
# 1/ Training data :

column_names = ['Model_name', 'input_shape', 'power', 'input_width', 'input_size', 'nb_params_conv','nb_params_bn', 'nb_params_fc', 'flops', 'nb_layers', 
                'nb_conv_layers', 'nb_bn_layers','nb_fc_layers', 'sum_activations', 'weighted_sum_neurons']
raw_dataset = pd.read_csv('./data/training_dataset.csv', names=column_names,
                      na_values = "?", comment='\t', sep=",", skipinitialspace=True)
train_dataset = raw_dataset.copy()
train_dataset.isna().sum()
train_dataset = train_dataset.dropna()

# 1/ Test data :

column_names = ['Model_name', 'input_shape', 'power', 'input_width', 'input_size', 'nb_params_conv','nb_params_bn', 'nb_params_fc', 'flops', 'nb_layers', 
                'nb_conv_layers', 'nb_bn_layers','nb_fc_layers', 'sum_activations', 'weighted_sum_neurons']
raw_dataset = pd.read_csv('./data/test_dim1.csv', names=column_names,
                      na_values = "?", comment='\t', sep=",", skipinitialspace=True)
test_dataset1 = raw_dataset.copy()
test_dataset1.isna().sum()
test_dataset1 = test_dataset1.dropna()

# 2/ Test data :

column_names = ['Model_name', 'input_shape', 'power', 'input_width', 'input_size', 'nb_params_conv','nb_params_bn', 'nb_params_fc', 'flops', 'nb_layers', 
                'nb_conv_layers', 'nb_bn_layers','nb_fc_layers', 'sum_activations', 'weighted_sum_neurons']
raw_dataset = pd.read_csv('./data/test_dim2.csv', names=column_names,
                      na_values = "?", comment='\t', sep=",", skipinitialspace=True)
test_dataset2 = raw_dataset.copy()
test_dataset2.isna().sum()
test_dataset2 = test_dataset2.dropna()

# 3/ Test data :

column_names = ['Model_name', 'input_shape', 'power', 'input_width', 'input_size', 'nb_params_conv','nb_params_bn', 'nb_params_fc', 'flops', 'nb_layers', 
                'nb_conv_layers', 'nb_bn_layers','nb_fc_layers', 'sum_activations', 'weighted_sum_neurons']
raw_dataset = pd.read_csv('./data/test_dim3.csv', names=column_names,
                      na_values = "?", comment='\t', sep=",", skipinitialspace=True)
test_dataset3 = raw_dataset.copy()
test_dataset3.isna().sum()
test_dataset3 = test_dataset3.dropna()

In [5]:
list_comp = ['input_width', 'input_size','nb_params_conv', 'nb_params_bn', 'nb_params_fc', 'nb_layers', 
                'nb_conv_layers', 'nb_bn_layers','nb_fc_layers', 'sum_activations', 'weighted_sum_neurons']

list_var = ['input_width', 'nb_params_bn', 'nb_params_fc', 
                'nb_conv_layers', 'nb_bn_layers','nb_fc_layers', 'weighted_sum_neurons']

train_dataset = select_train(list_var, train_dataset)
test_dataset1 = select_test(list_var, test_dataset1)
test_dataset2 = select_test(list_var, test_dataset2)
test_dataset3 = select_test(list_var, test_dataset3)

In [6]:
train_dataset.tail()

,Model_name,input_shape,power,input_size,nb_params_conv,flops,nb_layers,sum_activations
1288,basic_model_112_22,"(1, 112, 112, 3)",3675.0,37632,15145100,18946763197,43,17346904
1289,basic_model_56_5,"(1, 56, 56, 3)",3616.0,9408,1859668,11255068812,12,2242016
1290,basic_model_75_37,"(1, 75, 75, 3)",3575.0,16875,18939872,26945702640,64,9258839
1291,basic_model_200_50,"(1, 200, 200, 3)",3548.0,120000,20659419,19180879124,82,11849524
1292,basic_model_56_6,"(1, 56, 56, 3)",3531.0,9408,1727954,10997881289,14,7699280


In [7]:
Y = train_dataset.pop('power')
X = train_dataset

In [8]:
Y_e1 = test_dataset1.pop('power')
X_e1 = test_dataset1

Y_e2 = test_dataset2.pop('power')
X_e2 = test_dataset2

Y_e3 = test_dataset3.pop('power')
X_e3 = test_dataset3

In [9]:
X_train_model_names = X.pop('Model_name')
X_train_input_shapes = X.pop('input_shape')

In [10]:
X_test_model_names1 = X_e1.pop('Model_name')
X_test_input_shapes1 = X_e1.pop('input_shape')

X_test_model_names2 = X_e2.pop('Model_name')
X_test_input_shapes2 = X_e2.pop('input_shape')

X_test_model_names3 = X_e3.pop('Model_name')
X_test_input_shapes3 = X_e3.pop('input_shape')

In [11]:
y_test1 = Y_e1.values
y_test2 = Y_e2.values
y_test3 = Y_e3.values

In [12]:
polyn = PolynomialFeatures(degree=20)
xd = polyn.fit_transform(np.log(X.values))
clf_Ridge = linear_model.Ridge(alpha=6e-4, normalize=True,max_iter=1e5)
A = clf_Ridge.fit(xd,Y.values)

In [13]:
y_pred3 = clf_Ridge.predict(polyn.fit_transform(np.log(X_e3.values)))
y_pred2 = clf_Ridge.predict(polyn.fit_transform(np.log(X_e2.values)))
y_pred1 = clf_Ridge.predict(polyn.fit_transform(np.log(X_e1.values)))

In [36]:
y_test = y_test2
y_pred = y_pred2
test_models = X_test_model_names2.values
test_input_shapes = X_test_input_shapes2.values

In [37]:
print('Explained variance : ',explained_variance_score(y_test, y_pred))
print('R2_value : ',r2_score(y_test, y_pred))
print('Mean absolute error MAE :', mean_absolute_error(y_test, y_pred))
print('Root Mean squared error RMSE :', mean_squared_error(y_test, y_pred, squared=False))
print('Median absolute error MAD:', median_absolute_error(y_test, y_pred))

Explained variance :  0.9673843386794065
R2_value :  0.9645923120340861
Mean absolute error MAE : 134.4046134228357
Root Mean squared error RMSE : 187.4635946241569
Median absolute error MAD: 100.60137120547847


In [38]:
# Calculate mean absolute percentage error (MAPE)
errors = abs(y_pred-y_test)
mape = 100 * (errors / y_test) # Calculate and display accuracy
accuracy = 100 - np.mean(mape)
print('MAX MAPE: ', round(max(mape), 2), '%.')
print('MAPE:', round(np.mean(mape), 2), '%.')
print('Accuracy:', round(accuracy, 2), '%.')

MAX MAPE:  45.21 %.
MAPE: 6.51 %.
Accuracy: 93.49 %.


In [39]:
import csv
with open('./measured_vs_predicted_Poly_dim2.csv', 'w', newline='') as file : 
    writer = csv.writer(file)
    writer.writerow(['CNN_model', 'input_shape', 'model&shape', 'measured', 'predicted', 'absolute_deviation', 'MAPE (in %)'])
    for i in range(0, len(y_pred)) :
        f = ''+test_models[i]+'_'+test_input_shapes[i]
        writer.writerow([test_models[i], test_input_shapes[i], f, y_test[i], y_pred[i],abs(y_pred[i]-y_test[i]), ((abs(y_pred[i]-y_test[i]))/y_test[i])*100])